In [157]:
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import requests
import lxml
import webbrowser
import pandas as pd
import numpy as np
import re
import string
import gensim
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
from sklearn.metrics import accuracy_score, confusion_matrix, mean_squared_error, r2_score, classification_report
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from gensim.models import Word2Vec
import nltk
from nltk.corpus import stopwords
from nltk.tag import pos_tag
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\luisz\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\luisz\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\luisz\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\luisz\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [123]:
url = 'https://www.ign.com/reviews/movies'

headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}

html_text = requests.get(url, headers=headers).text

In [124]:
soup = BeautifulSoup(html_text, 'lxml')
main = soup.find('main', {"id": 'main-content'})
sections = main.section

In [125]:
print(sections)

<section class="grid jsx-949309242"><section class="box-wrapper jsx-537794787 grid-heading"><div class="jsx-651505178 filter-heading page-content"><h2 class="title1 jsx-4293198083 filter-title">Movie Reviews</h2></div><div class="jsx-1423006382 page-content"><div aria-label="Filters" class="jsx-3963919147 jsx-4243969252 tab-filter-bar jsx-3933161798" data-cy="filter-bar" role="tablist"><a aria-selected="false" class="tab-filter jsx-3710415076" href="/reviews" id="all" role="tab" tabindex="-1"><div class="interface jsx-153568585 bold small">All</div></a><a aria-selected="false" class="tab-filter jsx-3710415076" href="/reviews/games" id="games" role="tab" tabindex="-1"><div class="interface jsx-153568585 bold small">Games</div></a><a aria-selected="true" class="tab-filter jsx-3710415076 active" href="/reviews/movies" id="movies" role="tab" tabindex="0"><div class="interface jsx-153568585 bold small">Movies</div></a><a aria-selected="false" class="tab-filter jsx-3710415076" href="/reviews

In [126]:
figcaption_elements = soup.findAll('figcaption')

figcaption_values = []
for figcaption_element in figcaption_elements:
    figcaption_value = figcaption_element.text
    figcaption_values.append(figcaption_value)

In [127]:
figcaption_values

['8', '6', '7', '8', '7', '7', '4', '4', '6', '4', '6']

In [128]:
review_links = soup.find_all('a', class_='item-body')

hrefs = []
for review_link in review_links:
    href = review_link.get('href')
    hrefs.append(href)

In [129]:
hrefs

['/articles/its-a-wonderful-knife-review',
 '/articles/the-hunger-games-the-ballad-of-songbirds-and-snakes-review',
 '/articles/the-marvels-review-brie-larson-samuel-l-jackson-mcu',
 '/articles/divinity-review-bella-thorne-eddie-alcazar',
 '/articles/the-killer-review-david-fincher-michael-fassbender-netflix',
 '/articles/five-nights-at-freddys-review',
 '/articles/freelance-review-john-cena-alison-brie',
 '/articles/cobweb-geomijip-review-song-kang-ho-kim-jee-woon',
 '/articles/the-end-we-start-from-review-jodie-comer',
 '/articles/late-night-with-the-devil-review-david-dastmalchian']

In [130]:
base_url = 'https://www.ign.com'
data_from_urls = []

for href, score in zip(hrefs, figcaption_values):
    full_url = urljoin(base_url, href)
    response = requests.get(full_url, headers=headers)
    html_text = response.text
    soup = BeautifulSoup(html_text, 'lxml')
    page_title = soup.title.text
    paragraphs = soup.find_all('p', class_='jsx-3649800006')

    for paragraph in paragraphs:
        data_from_urls.append({'Title': page_title, 'Score': score, 'Paragraph': paragraph.text})

for data in data_from_urls:
    print(data)

{'Title': "It's a Wonderful Knife Review - IGN", 'Score': '8', 'Paragraph': 'It’s a Wonderful Knife is an excellent title for a Christmas slasher movie. So excellent, it’s surprising no one thought of it before. (That’s the sign of a good pun – it seems obvious, even inevitable.) Unfortunately, nothing else about the film matches that level of inspiration.'}
{'Title': "It's a Wonderful Knife Review - IGN", 'Score': '8', 'Paragraph': 'The opening sequence glows with the warm incandescent light of a Hallmark or Lifetime Christmas movie, introducing an idyllic town square full of smiling citizens in festive sweaters to match. Angel Falls is celebrating its annual Christmas tree lighting ceremony, and everything seems perfect – at least on the surface. (Behind the scenes, there’s some family drama going on, but that’s not really important.) Then “The Angel,” an anonymous killer in a white cape and smooth, featureless white mask, appears at a raucous Christmas Eve party attended by our oste

In [131]:
page_title

'Late Night with the Devil Review - IGN'

In [132]:
df = pd.DataFrame(data_from_urls)

In [133]:
df

,Title,Score,Paragraph
0,It's a Wonderful Knife Review - IGN,8,It’s a Wonderful Knife is an excellent title f...
1,It's a Wonderful Knife Review - IGN,8,The opening sequence glows with the warm incan...
2,It's a Wonderful Knife Review - IGN,8,"Fifteen minutes in, a title card appears on sc..."
3,It's a Wonderful Knife Review - IGN,8,"When she returns home, she finds that The Ange..."
4,It's a Wonderful Knife Review - IGN,8,Screenwriter Michael Kennedy isn’t beholden t...
...,...,...,...
71,Late Night with the Devil Review - IGN,4,The bulk of Late Night with the Devil is prese...
72,Late Night with the Devil Review - IGN,4,This should sound familiar to well-versed horr...
73,Late Night with the Devil Review - IGN,4,"Still, it’s an amusing enough spectacle to wat..."
74,Late Night with the Devil Review - IGN,4,But while Late Night with the Devil captures t...


In [134]:
df[df['Paragraph']==' ']

,Title,Score,Paragraph


In [135]:
df.replace(' ', np.nan, inplace=True)

In [136]:
df.dropna(subset=['Paragraph'], inplace=True)

In [137]:
df[df['Paragraph']==' ']

,Title,Score,Paragraph


In [138]:
def preprocess_text(text):
    text = text.lower()
    
    text = re.sub(r'\d+', '', text)
    text = re.sub(r'[^\w\s]', '', text)
    
    tokens = nltk.word_tokenize(text)
    
    return tokens

In [139]:
def remove_stopWords(tokens):
    stopWords = set(stopwords.words('english'))
    filtered_tokens = [words for words in tokens if words not in stopWords]
    return filtered_tokens

In [140]:
def lemmatization(tokens):
    lemmatizer = nltk.WordNetLemmatizer()
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens]
    return lemmatized_tokens

In [141]:
def remove_proper_names(tokens):
    tagged_tokens = pos_tag(tokens)
    filtered_tokens = [token[0] for token in tagged_tokens if token[1] != 'NNP']
    return filtered_tokens

In [142]:
def clean_text(text):
    tokens = preprocess_text(text)
    filtered_tokens = remove_stopWords(tokens)
    lemmatization_tokens = lemmatization(filtered_tokens)
    non_proper_name_tokens = remove_proper_names(filtered_tokens)
    clean_text = ' '.join(non_proper_name_tokens)
    return clean_text

In [143]:
df['cleaned_Paragraph'] = df['Paragraph'].apply(clean_text)

In [144]:
df

,Title,Score,Paragraph,cleaned_Paragraph
0,It's a Wonderful Knife Review - IGN,8,It’s a Wonderful Knife is an excellent title f...,wonderful knife excellent title christmas slas...
1,It's a Wonderful Knife Review - IGN,8,The opening sequence glows with the warm incan...,opening sequence glows warm incandescent light...
2,It's a Wonderful Knife Review - IGN,8,"Fifteen minutes in, a title card appears on sc...",fifteen minutes title card appears screen real...
3,It's a Wonderful Knife Review - IGN,8,"When she returns home, she finds that The Ange...",returns home finds angel caught previous chris...
4,It's a Wonderful Knife Review - IGN,8,Screenwriter Michael Kennedy isn’t beholden t...,screenwriter michael kennedy isnt beholden hol...
...,...,...,...,...
71,Late Night with the Devil Review - IGN,4,The bulk of Late Night with the Devil is prese...,bulk late night devil presented master recordi...
72,Late Night with the Devil Review - IGN,4,This should sound familiar to well-versed horr...,sound familiar wellversed horror nuts late nig...
73,Late Night with the Devil Review - IGN,4,"Still, it’s an amusing enough spectacle to wat...",still amusing enough spectacle watch unravel d...
74,Late Night with the Devil Review - IGN,4,But while Late Night with the Devil captures t...,late night devil captures smoky lates miasma c...


In [145]:
del df['Paragraph']

In [146]:
df

,Title,Score,cleaned_Paragraph
0,It's a Wonderful Knife Review - IGN,8,wonderful knife excellent title christmas slas...
1,It's a Wonderful Knife Review - IGN,8,opening sequence glows warm incandescent light...
2,It's a Wonderful Knife Review - IGN,8,fifteen minutes title card appears screen real...
3,It's a Wonderful Knife Review - IGN,8,returns home finds angel caught previous chris...
4,It's a Wonderful Knife Review - IGN,8,screenwriter michael kennedy isnt beholden hol...
...,...,...,...
71,Late Night with the Devil Review - IGN,4,bulk late night devil presented master recordi...
72,Late Night with the Devil Review - IGN,4,sound familiar wellversed horror nuts late nig...
73,Late Night with the Devil Review - IGN,4,still amusing enough spectacle watch unravel d...
74,Late Night with the Devil Review - IGN,4,late night devil captures smoky lates miasma c...


In [147]:
print(f'Worst reviews: \n{df.groupby("Title")["Score"].max().sort_values().head(3)}')
print()
print(f'Best reviews: \n{df.groupby("Title")["Score"].max().sort_values(ascending=False).head(3)}')


Worst reviews: 
Title
Cobweb (Geomijip) Review - IGN            4
Freelance Review - IGN                    4
Late Night with the Devil Review - IGN    4
Name: Score, dtype: object

Best reviews: 
Title
Divinity Review - IGN                   8
It's a Wonderful Knife Review - IGN     8
Five Nights at Freddy's Review - IGN    7
Name: Score, dtype: object


# Try Models

In [148]:
X = df['cleaned_Paragraph']
y = df['Score']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Convert text data to TF-IDF features
tfidf_vectorizer = TfidfVectorizer()
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)


### Support Vector Regression

In [158]:
sv_model = SVR(kernel='linear')  # There are many different kernels like 'linear', 'rbf', etc.

# Train the model
sv_model.fit(X_train_tfidf, y_train)

# Predict on the test set
y_pred_sv = sv_model.predict(X_test_tfidf)

#IGN Scores are Integers
y_pred_sv = np.around(y_pred_sv,0)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred_sv)
r_squared = r2_score(y_test, y_pred_sv)

print(f'Mean Squared Error: {mse}')
print(f'R-squared: {r_squared}')

ValueError: dtype='numeric' is not compatible with arrays of bytes/strings.Convert your data to numeric values explicitly instead.

In [87]:
print(y_pred_sv)
print(np.array(y_test))

[7. 6. 6. 7. 6. 6. 6. 6. 6. 6. 6. 6. 7. 6. 6. 7. 6. 7. 7. 6. 6. 6. 7.]
['8' '7' '6' '8' '7' '4' '6' '4' '4' '8' '4' '4' '7' '6' '4' '7' '6' '8'
 '7' '7' '8' '7' '7']


### Random Forest

In [95]:
rf_model = RandomForestRegressor(n_estimators=150, random_state=42, max_depth=10)

# Train the model
rf_model.fit(X_train_tfidf, y_train)

# Predict on the test set
y_pred_rf = rf_model.predict(X_test_tfidf)

#IGN Scores are Integers
y_pred_sv = np.around(y_pred_rf,0)

# Evaluate the model
mse_rf = mean_squared_error(y_test, y_pred_rf)
r_squared_rf = r2_score(y_test, y_pred_rf)

print(f'Mean Squared Error (Random Forest): {mse_rf}')
print(f'R-squared (Random Forest): {r_squared_rf}')

Mean Squared Error (Random Forest): 1.571517527436379
R-squared (Random Forest): 0.2833338172294443


In [160]:
print(np.around(y_pred_rf,0))
print(np.array(y_test))

[7. 7. 7. 7. 7. 7. 6. 7. 5. 7. 4. 4. 7. 7. 6. 7. 7. 7. 7. 6. 6. 6. 7.]
['8' '7' '6' '8' '7' '4' '6' '4' '4' '8' '4' '4' '7' '6' '4' '7' '6' '8'
 '7' '7' '8' '7' '7']


## Grid Searches

### SVR

In [161]:
# Define the parameter grid for SVR
svr_param_grid = {'C': [0.1, 1, 10],
                  'kernel': ['linear', 'rbf', 'poly'],
                  'gamma': ['scale', 'auto']}

# Create the SVR model
svr_model = SVR()

# Create GridSearchCV for SVR
svr_grid_search = GridSearchCV(svr_model, svr_param_grid, cv=5, scoring='neg_mean_squared_error')
svr_grid_search.fit(X_train_tfidf, y_train)

# Get the best parameters and model
best_svr_params = svr_grid_search.best_params_
best_svr_model = svr_grid_search.best_estimator_

# Predict on the test set
y_pred_svr = best_svr_model.predict(X_test_tfidf)

# Evaluate the model
mse_svr = mean_squared_error(y_test, y_pred_svr)
r_squared_svr = r2_score(y_test, y_pred_svr)

print("Best parameters for SVM:", best_svr_params)
print(f'Mean Squared Error (SVM): {mse_svr}')
print(f'R-squared (SVM): {r_squared_svr}')


Best parameters for SVM: {'C': 10, 'gamma': 'scale', 'kernel': 'linear'}
Mean Squared Error (SVM): 1.3513823254582094
R-squared (SVM): 0.3837230602005235


### Random Forest

In [100]:
# Define the parameter grid for Random Forest
rf_param_grid = {'n_estimators': [50, 100, 150],
                 'max_depth': [None, 10, 20, 30],
                 'min_samples_split': [2, 5, 10],
                 'min_samples_leaf': [1, 2, 4]}

# Create the Random Forest model
rf_model = RandomForestRegressor(random_state=42)

# Create GridSearchCV for Random Forest
rf_grid_search = GridSearchCV(rf_model, rf_param_grid, cv=5, scoring='neg_mean_squared_error')
rf_grid_search.fit(X_train_tfidf, y_train)

# Get the best parameters and model
best_rf_params = rf_grid_search.best_params_
best_rf_model = rf_grid_search.best_estimator_

# Predict on the test set
y_pred_rf_gs = best_rf_model.predict(X_test_tfidf)

# Evaluate the model
mse_rf_gs = mean_squared_error(y_test, y_pred_rf_gs)
r_squared_rf_gs = r2_score(y_test, y_pred_rf_gs)

print("Best parameters for Random Forest:", best_rf_params)
print(f'Mean Squared Error (Random Forest): {mse_rf_gs}')
print(f'R-squared (Random Forest): {r_squared_rf_gs}')


Best parameters for Random Forest: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100}
Mean Squared Error (Random Forest): 1.5773304347826085
R-squared (Random Forest): 0.2806829310344827


### Naive Bayes

In [149]:
nb = Pipeline([('vect', CountVectorizer()),
               ('tfidf', TfidfTransformer()),
               ('clf', MultinomialNB()),
              ])
nb.fit(X_train, y_train)

Pipeline(steps=[('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('clf', MultinomialNB())])

In [150]:
df['Score'].unique()

array(['8', '6', '7', '4'], dtype=object)

In [151]:
y_pred_nb = nb.predict(X_test)

print('accuracy %s' % accuracy_score(y_pred_nb, y_test))
print(classification_report(y_test, y_pred_nb,target_names= df['Score'].unique()))

accuracy 0.6086956521739131
              precision    recall  f1-score   support

           8       0.86      1.00      0.92         6
           6       0.00      0.00      0.00         4
           7       0.50      1.00      0.67         8
           4       0.00      0.00      0.00         5

    accuracy                           0.61        23
   macro avg       0.34      0.50      0.40        23
weighted avg       0.40      0.61      0.47        23



C:\Users\luisz\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\luisz\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\luisz\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
print(np.around(y_pred_rf,0))
print(np.array(y_test))

### Linear Support Vector Machine

In [152]:
svm = Pipeline([('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('clf', SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, random_state=42, max_iter=5, tol=None)),
               ])
svm.fit(X_train, y_train)

Pipeline(steps=[('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('clf',
                 SGDClassifier(alpha=0.001, max_iter=5, random_state=42,
                               tol=None))])

In [153]:
y_pred_svm = svm.predict(X_test)

print('accuracy %s' % accuracy_score(y_pred_svm, y_test))
print(classification_report(y_test, y_pred_svm,target_names=df['Score'].unique()))

accuracy 0.9130434782608695
              precision    recall  f1-score   support

           8       0.86      1.00      0.92         6
           6       1.00      1.00      1.00         4
           7       0.89      1.00      0.94         8
           4       1.00      0.60      0.75         5

    accuracy                           0.91        23
   macro avg       0.94      0.90      0.90        23
weighted avg       0.92      0.91      0.91        23



In [155]:
print(np.around(y_pred_svm,0))
print(np.array(y_test))

['8' '7' '6' '8' '7' '4' '6' '4' '4' '8' '4' '4' '7' '6' '4' '7' '6' '7'
 '7' '7' '4' '7' '7']
